In [54]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [55]:
import glob

In [56]:
chessboard_size = (9,6)

### 1. Initialize arrays to save both grid points and image points

In [57]:
obj_points = [] # 3D coordinates of our grid
img_points = [] # 3D points of the image

In [58]:
# Create array for chessboard coordinates
objp = np.zeros((np.prod(chessboard_size),3), dtype = np.float32) # Array of (9*6,3) with float zeros

In [59]:
# Magic happens... we need an index later on
objp[:,:2] = np.mgrid[0:chessboard_size[0],0:chessboard_size[1]].T.reshape(-1,2)

### 2. Find chessboard corners on all images and store them (after some processing)

In [60]:
calib_imgs = glob.glob("../opencv-np-workshop/data/calib/*")

In [61]:
calib_imgs

['../opencv-np-workshop/data/calib\\P2280001.JPG',
 '../opencv-np-workshop/data/calib\\P2280002.JPG',
 '../opencv-np-workshop/data/calib\\P2280003.JPG',
 '../opencv-np-workshop/data/calib\\P2280004.JPG',
 '../opencv-np-workshop/data/calib\\P2280005.JPG',
 '../opencv-np-workshop/data/calib\\P2280006.JPG',
 '../opencv-np-workshop/data/calib\\P2280007.JPG',
 '../opencv-np-workshop/data/calib\\P2280008.JPG']

In [62]:
for img_path in calib_imgs:
    img = cv2.imread(img_path, 0)
    #img = cv2.pyrDown(img) # Downsample for speed
    ret, corners = cv2.findChessboardCorners(img, chessboard_size, None)
    
    if ret:
        # Takes some time, so better to log something
        print("Chessboard detected")
        print(img_path)
        
        # Increase subpixel accuracy of detection --> Reccommended in docs
        criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        
        # Refine corner location
        corners = cv2.cornerSubPix(img, corners, (11,11), (-1,-1), criteria)
        
        # Store corners + index
        obj_points.append(objp)
        img_points.append(corners)        

Chessboard detected
../opencv-np-workshop/data/calib\P2280001.JPG
Chessboard detected
../opencv-np-workshop/data/calib\P2280002.JPG
Chessboard detected
../opencv-np-workshop/data/calib\P2280003.JPG
Chessboard detected
../opencv-np-workshop/data/calib\P2280004.JPG
Chessboard detected
../opencv-np-workshop/data/calib\P2280005.JPG
Chessboard detected
../opencv-np-workshop/data/calib\P2280006.JPG
Chessboard detected
../opencv-np-workshop/data/calib\P2280007.JPG
Chessboard detected
../opencv-np-workshop/data/calib\P2280008.JPG


### 3. Calibrate camera

In [63]:
ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
                                        obj_points, 
                                        img_points, 
                                        img.shape[::-1],
                                        None,
                                        None
                                    )

In [64]:
K

array([[3.57276978e+03, 0.00000000e+00, 2.11492730e+03],
       [0.00000000e+00, 3.58000354e+03, 1.69882805e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [65]:
img.shape

(3216, 4288)

In [66]:
# Refine the camera matrix
h, w = img.shape

In [67]:
K_opt, roi = cv2.getOptimalNewCameraMatrix(K, dist, (w,h), 1, (w,h)) 
# 1 = alpha parameter (no point deletion)

In [68]:
K_opt

array([[3.38229004e+03, 0.00000000e+00, 2.11738348e+03],
       [0.00000000e+00, 3.43007544e+03, 1.73542327e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

### 4. Measure calibration error

In [70]:
mean_error = 0 
for i in range(len(img_points)):
    # virtual picture of the chessboard
    img_points2, _ = cv2.projectPoints(obj_points[i], rvecs[i], tvecs[i], K, dist) 
    error = cv2.norm(img_points[i], img_points2, cv2.NORM_L2)/len(img_points2)
    
    mean_error += error # averar error in instance
print("Error: ", mean_error/len(obj_points))
    

Error:  0.1264159553783108


In [71]:
np.save("../opencv-np-workshop/data/camera_params/K", K)